In [1]:
'''
Trying to understand how to use embedding over one hot
'''

'\nTrying to understand how to use embedding over one hot\n'

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:

'''
Connect torch to processor
'''
if (torch.cuda.is_available()):
    torch.cuda.empty_cache()

    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

    # Set the desired GPU (e.g., the first GPU)
    torch.cuda.set_device(0)
    print(f"Using GPU: {torch.cuda.current_device()} - {torch.cuda.get_device_name(torch.cuda.current_device())}")

    # Set device
    device = torch.device("cuda:0")

    print("Successful GPU connect")
else:
    device = torch.device('cpu')
    print("Using CPU")

GPU 0: NVIDIA GeForce MX150
Using GPU: 0 - NVIDIA GeForce MX150
Successful GPU connect


In [4]:
df = pd.read_csv(r"./lung_cancer_mortality_data_large.csv")
df.head()

,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,1,64.0,Female,Germany,2016-04-05,Stage III,No,Never Smoked,31.1,257,1,1,0,0,Combined,2017-11-13,0
1,2,50.0,Male,Czech Republic,2023-04-20,Stage III,Yes,Passive Smoker,25.9,208,1,0,0,0,Radiation,2024-09-02,0
2,3,65.0,Male,Romania,2023-04-05,Stage IV,No,Never Smoked,18.9,193,0,0,0,0,Surgery,2024-10-08,0
3,4,51.0,Female,Latvia,2016-02-05,Stage III,Yes,Former Smoker,34.6,249,1,1,1,0,Surgery,2017-05-08,1
4,5,37.0,Male,Greece,2023-11-29,Stage I,Yes,Never Smoked,40.2,262,0,0,0,0,Chemotherapy,2025-05-03,0


In [5]:
# FIXME: Need to incorperate dates
try:
    df = df.drop(columns=['id', 'end_treatment_date', 'diagnosis_date'])
except KeyError:
    pass

# Binary
encoder = LabelEncoder()
df['family_history'] = encoder.fit_transform(df['family_history'])
df['gender'] = encoder.fit_transform(df['gender'])

# One hot
treatment_types = ['Radiation', 'Surgery', 'Chemotherapy']
for treatement in treatment_types:
    df[treatement] = 0

df.loc[df['treatment_type'] == 'Combined', treatment_types] = 1

for treatment in treatment_types:
    df.loc[df['treatment_type'] == treatment, treatment] = 1

df = df.drop(columns=['treatment_type'])

# Embedded
df['country_code'] = df['country'].astype('category').cat.codes
num_countries = len(df['country'].unique())
country_codes = torch.tensor(df['country_code'].values, dtype=torch.long)
df = df.drop(columns=['country'])

# Ordered (also embedded)
smoking_status_order = ['Never Smoked', 'Passive Smoker', 'Former Smoker', 'Current Smoker']
df['smoking_status'] = pd.Categorical(df['smoking_status'], categories=smoking_status_order, ordered=True)
df['smoking_status'] = df['smoking_status'].cat.codes
cancer_stage_order = ['Stage I', 'Stage II', 'Stage III', 'Stage IV']
df['cancer_stage'] = pd.Categorical(df['cancer_stage'], categories=cancer_stage_order, ordered=True)
df['cancer_stage'] = df['cancer_stage'].cat.codes

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3250000 entries, 0 to 3249999
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   age                float64
 1   gender             int32  
 2   cancer_stage       int8   
 3   family_history     int32  
 4   smoking_status     int8   
 5   bmi                float64
 6   cholesterol_level  int64  
 7   hypertension       int64  
 8   asthma             int64  
 9   cirrhosis          int64  
 10  other_cancer       int64  
 11  survived           int64  
 12  Radiation          int64  
 13  Surgery            int64  
 14  Chemotherapy       int64  
 15  country_code       int8   
dtypes: float64(2), int32(2), int64(9), int8(3)
memory usage: 306.8 MB


In [6]:
df.head()

,age,gender,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,survived,Radiation,Surgery,Chemotherapy,country_code
0,64.0,0,2,0,0,31.1,257,1,1,0,0,0,1,1,1,10
1,50.0,1,2,1,1,25.9,208,1,0,0,0,0,1,0,0,5
2,65.0,1,3,0,0,18.9,193,0,0,0,0,0,0,1,0,22
3,51.0,0,2,1,2,34.6,249,1,1,1,0,1,0,1,0,15
4,37.0,1,0,1,0,40.2,262,0,0,0,0,0,0,0,1,11


In [7]:
scaler = StandardScaler()
numerical_features = ['age', 'bmi', 'cholesterol_level']
df[numerical_features] = scaler.fit_transform(df[numerical_features])

df.head()

,age,gender,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,survived,Radiation,Surgery,Chemotherapy,country_code
0,0.899921,0,2,0,0,0.072537,0.537704,1,1,0,0,0,1,1,1,10
1,-0.499908,1,2,1,1,-0.548619,-0.590068,1,0,0,0,0,1,0,0,5
2,0.999909,1,3,0,0,-1.384790,-0.935305,0,0,0,0,0,0,1,0,22
3,-0.399921,0,2,1,2,0.490622,0.353578,1,1,1,0,1,0,1,0,15
4,-1.799750,1,0,1,0,1.159559,0.652782,0,0,0,0,0,0,0,1,11


In [8]:
# Move 'survived' to end so the indices make scence after its droped
df = df[[c for c in df if c not in ['survived']] + ['survived']]

X = df.drop('survived', axis=1)
y = df['survived']

X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)

print(X.shape)

torch.Size([3250000, 15])


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

cat_indices = [df.columns.get_loc(col) for col in ['country_code', 'cancer_stage', 'smoking_status']]
num_indices = [i for i in range(X.shape[1]) if i not in cat_indices]

X_train_cat = X_train[:, cat_indices].long()
X_train_num = X_train[:, num_indices]

X_test_cat = X_test[:, cat_indices].long()
X_test_num = X_test[:, num_indices]

In [10]:
class Model(nn.Module):
    def __init__(self, num_countries, num_cancer_stages, num_smoking_statuses, embedding_dim, num_numerical_features):
        super(Model, self).__init__()
        # Embeddings. Cancer stage and smoking status could be one hot
        self.embedding_country = nn.Embedding(num_countries, embedding_dim)
        self.embedding_cancer_stage = nn.Embedding(num_cancer_stages, embedding_dim)
        self.embedding_smoking_status = nn.Embedding(num_smoking_statuses, embedding_dim)

        self.fc1 = nn.Linear(3 * embedding_dim + num_numerical_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_cat, x_num):
        x_country = self.embedding_country(x_cat[:, 0])
        x_cancer_stage = self.embedding_cancer_stage(x_cat[:, 1])
        x_smoking_status = self.embedding_smoking_status(x_cat[:, 2])
        
        x_cat_combined = torch.cat([x_country, x_cancer_stage, x_smoking_status], dim=1)
        
        x = torch.cat([x_cat_combined, x_num], dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [11]:
num_countries = len(df['country_code'].unique())
num_cancer_stages = len(df['cancer_stage'].unique())
num_smoking_statuses = len(df['smoking_status'].unique())
embedding_dim = 10  # Wrong?
num_numerical_features = X.shape[1] - 3

mod = Model(num_countries, num_cancer_stages, num_smoking_statuses, embedding_dim, num_numerical_features)
mod.to(device)

# Move all data to GPU (Moving by batch was too slow)
X_train_cat = X_train_cat.to(device)
X_train_num = X_train_num.to(device)
y_train = y_train.to(device)

X_test_cat = X_test_cat.to(device)
X_test_num = X_test_num.to(device)
y_test = y_test.to(device)

batch_size = 1024

train_dataset = TensorDataset(X_train_cat, X_train_num, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_cat, X_test_num, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mod.parameters(), lr=0.001)

In [12]:
# Training loop
epochs = 10
for epoch in range(epochs):
    mod.train()
    running_loss = 0.0
    for X_cat_batch, X_num_batch, y_batch in train_loader:
        # Move data to GPU
        # X_cat_batch, X_num_batch, y_batch = X_cat_batch.to(device), X_num_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = mod(X_cat_batch, X_num_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluate the model
mod.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X_cat_batch, X_num_batch, y_batch in test_loader:        
        outputs = mod(X_cat_batch, X_num_batch)
        predicted = outputs.round()
        total += y_batch.size(0)
        correct += (predicted.squeeze() == y_batch).sum().item()
    
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')

# 1m 32.7s on cpu for 10 epochs

Epoch [5/10], Loss: 0.5263
Epoch [10/10], Loss: 0.5262
Accuracy: 0.7800


In [13]:
torch.cuda.empty_cache()